## Machine Learning & Deep Learning: GridSearch

Ein kleines Add-On, in dem ich Ihnen zeige, wie man die Grid Search und Random Search zur Optimierung der Hyperparameter anwendet. 

### Schritt 1: Importiere relevante Packages

In [ ]:
import pandas as pd 
import numpy as np
import helpers

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

# Validation libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, precision_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score


# Display plots inside the notebook
%matplotlib inline
sns.set(style="white", palette="pastel")

# Ignore warning related to pandas_profiling
import warnings
warnings.filterwarnings('ignore') 

from pathlib import Path

# Display all dataframe columns in outputs (it has 63 columns, which is wider than the notebook)
# This sets it up to display with a horizontal scroll instead of hiding the middle columns
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_colwidth', -1)

## Schritt 2: Train und Test Datensatz festlegen

In [ ]:
# mit der Pickle Funktion packen wir den fertig bearbeiteten Dataframe in eine Datei, die wir nächste Woche einfach 
# importieren können. 
X = pd.read_pickle("X_df.pkl")
y = pd.read_pickle("y_df.pkl")

In [ ]:
seed = 5 # wenn wir den seed festlegen können wir den random-split später reproduzieren
test_size = 0.33 # d.h. 33% des Datensatzes werden zufällig ausgewählt und dem Test- Datensatz zugeordnet.

# Durch den Train Test Split teilen wir den Datensatz in einen Train-Datensatz und einen Test-Datensatz auf. 
# Das Modell wird auf dem Train-Datensatz trainiert und am Test-Datensatz getestet.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#### Vorbereitung zur Evaluierung von Modellen:
1. An den gleichen Daten trainieren und testen: Wir wissen nicht, ob das Modell auch für andere Daten gute Vorhersagen trifft. 
2. **Train/Test Split**: Teilt den Datensatz in zwei Teile auf, sodass an unterschiedlichen Daten gelernt und getestet wird. Bessere Schätzung der Out-of-Sample-Performance (=Generalisierung) als 1.

### 2.1 K-fold Cross-Validation

Bei der K-fold Cross-Validation wir der Datensatz in K verschiedene Subsamples (=folds) unterteilt. Jeder fold wird dabei einmal als Test-Datensatz benutzt (während die restlichen **K-1** folds zum Training genutzt werden). So kann die Modell-Güte an verschiedenen Datensätzen getestet werden und ist somit objektiver. Das Ergebnis sind **K** Accuracy-Scores, aus denen i.d.R. der Mittelwert gebildet wird. 

In [ ]:
from sklearn.model_selection import cross_val_score

### 2.2 Grid Search zur Optimierung der Hyperparameter

Zur Wiederholung: Ein **Hyperparameter** ist eine Eigenschaft des Modells, die vor dem Training manuell festgelegt wird und nicht aus den Daten gelernt werden kann. Beispiele sind C und gamma bei Support Vector Machines oder die Anzahl an Hidden Layers in Neural Networks. Im Gegensatz dazu werden **Model Parameter** aus den Daten geschätzt. Beispiele dafür sind die Support Vectors bei Support Vector Machines oder beta-Koeffizienten in der Logistischen Regression. 

Beim **Grid Search** wird eine Liste von möglichen Hyperparametern festgelegt. Dann werden systematisch **alle** Hyperparameter miteinander kombiniert und getestet, bis der beste Accuracy Score gefunden wurde. 

In [ ]:
from sklearn.model_selection import GridSearchCV

## Schritt 3: Modelle 

### 3.1. Logistische Regression

### Grid Search bei Logistic Regression

Nun wenden wir Grid Search bei der Logistischen Regression an. Es werden verschiedene Werte für C und für die **penalty terms** (mehr dazu hier: https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c) angelegt und miteinander kombiniert. GridSearchCV wendet automatisch auch Cross Validation an.

In [ ]:
log_model = LogisticRegression()

# Liste an Hyperparametern
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}

# Modell anlegen
grid_log_model = GridSearchCV(log_model, param_grid = grid_values,scoring = 'accuracy', cv=10)

# Modell trainieren
grid_log_model.fit(X, y)

Show best Hyperparameters and best Cross Validation Score 

In [ ]:
print("Best hyperparameters: ", grid_log_model.best_params_)

print("\nBest Cross Validation Score      :", grid_log_model.best_score_)

### 3. 2. Grid Search bei Support Vector Classifiers

Bei der Grid Search für Support Vector Classifier können wir die Kernels (also z.B. linear, RBF) auch als Hyperparameter Liste angeben. Wenn es bei Ihnen zu lange dauert (~ mehr als 10 Minuten) können Sie gerne die Random Search ausprobieren. 

In [ ]:
# Legen Sie einen Support Vector Classifier ohne weitere Hyperparameter Angaben an

svc = SVC(probability=True)

# Liste an Hyperparametern festlegen
grid_values = {'C':[0.001,.009,0.01,.09,1,5,10,25], 
               'kernel':['linear', 'rbf'], 
               'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 100]}

# Grid Search festlegen. Das Ausführen kann eine Weile dauern. 

grid_svc_model = GridSearchCV(svc,param_grid=grid_values,scoring='accuracy',n_jobs=1,cv=10)


# Trainieren Sie den Classifier
grid_svc_model.fit(X,y)

In [ ]:
print("Best hyperparameters: ", grid_svc_model.best_params_)

print("\nBest Cross Validation Score      :", grid_svc_model.best_score_)

### 3. 2. Random Search bei Support Vector Classifiers

Die Grid Search kann bei einer größeren Anzahl an Hyperparametern sehr lange dauern. Etwas effizienter ist dabei die Random Search, bei der die Hyperparameter zufällig miteinander kombiniert werden. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
random_svc_model = RandomizedSearchCV(svc,param_distributions=grid_values,scoring='accuracy',n_jobs=1,cv=10)

In [ ]:
random_svc_model.fit(X,y)

In [ ]:
print("Best hyperparameters: ", random_svc_model.best_params_)

print("\nBest Cross Validation Score      :", random_svc_model.best_score_)

### Weitere Ressourcen:

- Chris Albon ist Autor von dem gern genutzten "Machine Learning with Python Cookbook". Auf seiner Seite finden sich ziemlich viele und gute Ressourcen https://chrisalbon.com/